# Notebook Import

In [159]:
import pandas as pd
import numpy as np

# Constants

In [221]:
path_root = 'C:/Users/97798/OneDrive/Desktop/Folder/M Learning/06 pre-process text data/SpamData/01_Processing/'

path_root_text = 'C:/Users/97798/OneDrive/Desktop/Folder/M Learning/06 pre-process text data/SpamData/02_Training/'

path_root_testing = 'C:/Users/97798/OneDrive/Desktop/Folder/M Learning/06 pre-process text data/SpamData/03_Testing/'
# saving the text file

TRAINING_DATA_FILE = path_root_text + 'train_data.txt'
TEST_DATA_FILE = path_root_text + 'train_test_data.txt'

# Saving all the data of spam and non spam emails 
TOKEN_SPAM_PROB_FILE = path_root_testing + 'prob-spam.txt'
TOKEN_HAM_PROB_FILE = path_root_testing + 'prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = path_root_testing + 'prob-all-tokens.txt'


TEST_FEATURE_MATRIX = path_root_testing + 'test-features.txt'
TEST_TARGET_FILE = path_root_testing + 'test-target.txt'

VOCAB_SIZE = 2500


# Read and load Features from .txt files into Numpy Array

In [161]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [162]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [163]:
sparse_train_data[:5]

array([[ 0,  2,  1,  1],
       [ 0,  3,  1,  2],
       [ 0,  4,  1,  1],
       [ 0,  7,  1,  3],
       [ 0, 11,  1,  1]])

In [164]:
sparse_test_data[-5:]

array([[5793, 2346,    0,    2],
       [5793, 2354,    0,    1],
       [5793, 2360,    0,    4],
       [5793, 2418,    0,    3],
       [5793, 2454,    0,    1]])

In [165]:
print('Nr of rows in training file',sparse_train_data.shape[0])
print('Nr of rows in test file',sparse_test_data.shape[0])

Nr of rows in training file 258381
Nr of rows in test file 117785


In [166]:
print('Nr of emails in traning file',np.unique(sparse_train_data[:,0]).size)


Nr of emails in traning file 4013


In [167]:
print('Nr of emails in test file',np.unique(sparse_test_data[:,0]).size)

Nr of emails in test file 1724


## How to Create an Empty DataFrame

In [168]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [169]:
len(column_names)

2502

In [170]:
index_names = np.unique(sparse_train_data[:, 0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [171]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value=0, inplace=True)

In [172]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [173]:
sparse_train_data[10:13]

array([[ 0, 31,  1,  1],
       [ 0, 33,  1,  1],
       [ 0, 44,  1,  1]])

In [174]:
sparse_train_data[10][3]

1

# Create a Full Matrix from a Sparse Matrix

In [175]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Return a pandas dataframe. 
    Keyword arguments:
    sparse_matrix -- numpy array
    nr_words -- size of the vocabulary. Total number of tokens. 
    doc_idx -- position of the document id in the sparse matrix. Default: 1st column
    word_idx -- position of the word id in the sparse matrix. Default: 2nd column
    cat_idx -- position of the label (spam is 1, nonspam is 0). Default: 3rd column
    freq_idx -- position of occurrence of word in sparse matrix. Default: 4th column
    """
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurrence
    
    full_matrix.set_index('DOC_ID', inplace=True)
    return full_matrix
    

In [176]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

Wall time: 9.22 s


In [177]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,14,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [178]:
full_train_data.tail()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
5789,0,3,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,1,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,3,1,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5794,0,1,1,1,0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
5795,0,3,4,2,0,5,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0


In [179]:
# Challenge: Calculate the probability of spam - the percent of spam messages in the training
# dataset. Store this value in a variable called prob_spam

In [180]:
full_train_data.CATEGORY.size

4013

In [181]:
full_train_data.CATEGORY.sum()

1248

In [182]:
prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size

In [183]:
print('Probability of spam is', prob_spam)

Probability of spam is 0.310989284824321


## Total Number of Words/Tokens

In [184]:
# loc is for selecting the rows - [:,# sabai rows haru select gara baneko and != # baneko categiry slect nagara]
full_train_features = full_train_data.loc[:,full_train_data.columns !='CATEGORY']

In [185]:
email_lengths = full_train_features.sum(axis = 1)
email_lengths.shape

(4013,)

In [186]:
email_lengths[:5]

DOC_ID
0     87
1     53
2     40
3    183
4     43
dtype: int64

In [187]:
total_wc = email_lengths.sum()

In [188]:
total_wc

429247

## Number of Tokens in Spam & Ham Emails

**Challenge** Create a subset of the email_lengths series that only contains the spam messages. Call the subset spam_lengths. Then count the total number of words that occur in spam emails. 

Do the same for the non-spam emails. Create a subset called ham_lengths. Then count the total number of words that occur in the ham emails. 

In [189]:
spam_lengths = email_lengths[full_train_data.CATEGORY ==1]

In [190]:
spam_lengths.shape

(1248,)

In [191]:
spam_wc = spam_lengths.sum()
spam_wc

176355

In [192]:
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
ham_lengths.shape

(2765,)

In [193]:
email_lengths.shape[0] - spam_lengths.shape[0] -ham_lengths.shape[0] 

0

In [194]:
nonspam_wc = ham_lengths.sum()
nonspam_wc

252892

In [195]:
spam_wc + nonspam_wc - total_wc

0

In [196]:
# Challenge: Can you figure out if spam emails or non-spam emails tend to be longer? Which
# category contains more tokens? Take a guess and then verify your intuition using the dataset

In [197]:
print('Average nr of words in spam emails {:.0f}'.format(spam_wc / spam_lengths.shape[0]))
print('Average nr of words in ham emails {:.3f}'.format(nonspam_wc / ham_lengths.shape[0]))

Average nr of words in spam emails 141
Average nr of words in ham emails 91.462


## Summing the Tokens Occuring in Spam

In [198]:
full_train_features.shape

(4013, 2500)

In [199]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]

In [200]:
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [201]:
train_spam_tokens.tail()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1885,1,0,0,2,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1887,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1890,2,0,0,0,1,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1895,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [202]:
train_spam_tokens.shape

(1248, 2500)

In [203]:
summed_spam_tokens = train_spam_tokens.sum(axis = 0)+1

In [204]:
summed_spam_tokens.shape

(2500,)

In [205]:
summed_spam_tokens.tail()

2495    11
2496     7
2497    10
2498    30
2499    11
dtype: int64

In [206]:
# Challenge: Repeat this process for the ham messages. Sum the tokens that occur in the 
# non-spam messages. Store the values in a variable called summed_ham_tokens. 

## Summing the Tokens Occuring in Ham

In [207]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
train_ham_tokens.shape

(2765, 2500)

In [208]:
train_ham_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1896,1,2,1,0,1,1,2,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1898,2,1,0,1,1,0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1899,2,1,1,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1900,3,4,0,1,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1901,2,2,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [209]:
train_ham_tokens.tail()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
5789,3,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,1,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,3,1,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5794,1,1,1,0,0,1,2,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5795,3,4,2,0,5,0,3,0,0,4,...,0,0,0,0,0,0,0,0,0,0


In [210]:
summed_ham_tokens = train_ham_tokens.sum(axis = 0)+1
summed_ham_tokens.shape

(2500,)

In [211]:
summed_ham_tokens.head()

0    5484
1    2590
2    2045
3     938
4    1612
dtype: int64

In [212]:
summed_ham_tokens.tail()

2495    23
2496    26
2497    21
2498     3
2499    21
dtype: int64

In [213]:
train_ham_tokens[2499].sum() + 1

21

## P(Token | Spam) - Probability that a Token Occurs given the Email is Spam

In [214]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)
prob_tokens_spam[:5]

0    0.012183
1    0.005228
2    0.006804
3    0.011305
4    0.006816
dtype: float64

In [215]:
prob_tokens_spam.sum()

1.0

## P(Token | Ham) - Probability that a Token Occurs given the Email is Nonspam

In [216]:
prob_tokens_ham = summed_ham_tokens / (nonspam_wc + VOCAB_SIZE)

In [217]:
prob_tokens_ham.sum()

1.0

# P(Token) - Probability that Token Occurs 

In [218]:
prob_tokens_all = full_train_features.sum(axis=0) / total_wc

In [219]:
prob_tokens_all.sum()

1.0

# Save the Trained Model

In [220]:
np.savetxt(TOKEN_SPAM_PROB_FILE,prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE,prob_tokens_ham)
np.savetxt(TOKEN_ALL_PROB_FILE,prob_tokens_all)

# Prepare Test Data

**Challenge:** Create a full matrix from the sparse_test_data. Time the function call. How long does it take? Separate the features and the target values. Save these as separate .txt files: a TEST_TARGET_FILE and a TEST_FEATURE_MATRIX file. 

In [222]:
sparse_test_data.shape

(117785, 4)

In [223]:
%%time
full_test_data = make_full_matrix(sparse_test_data, nr_words=VOCAB_SIZE)

Wall time: 6.28 s


In [224]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [225]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, X_test)